# TODO: 
- align methodology (col names etc etc) exactly to new data gathering process

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine  
import datetime 
import yfinance as yf
import warnings
# import norgatedata
warnings.filterwarnings('ignore')

# load metadata and relevant data
sharadar_sectors = pd.read_csv("C:/Users/lbianculli/backup/quant_analysis/dev/data/maps/sharadar_meta2.csv")
price_data = pd.read_csv("C:/Users/lbianculli/dev/us_equities/norgate/nogate_pricing.csv").drop(["Unnamed: 0"], axis=1)
all_macro_data = pd.read_csv("C:/Users/lbianculli/dev/us_equities/macro/macro_data.csv").drop(["Unnamed: 0"], axis=1)

# fundamental data
all_fundamental_data = pd.read_csv("C:/Users/lbianculli/dev/us_equities/ML_categorization/quandl_fundamentals.csv")
all_fundamental_data = all_fundamental_data.loc[all_fundamental_data["dimension"] == "MRQ"]

for c in all_fundamental_data.columns:
    try:
        all_fundamental_data[c] = all_fundamental_data[c].astype(float)
    except Exception as e:
        pass


In [2]:
# get nearest business day so we can merge later
from pandas.tseries.offsets import BDay

price_data["date_dt"] = pd.to_datetime(price_data["date_dt"], format='%Y-%m-%d')

# set up id
all_fundamental_data["date_dt"] = pd.to_datetime(all_fundamental_data["calendar_date"], format='%Y-%m-%d')
all_fundamental_data["date"] = all_fundamental_data["date_dt"].dt.strftime(date_format="%Y-%m-%d")
all_fundamental_data["year"] = all_fundamental_data["date_dt"].dt.year
all_fundamental_data["month"] = all_fundamental_data["date_dt"].dt.month
all_fundamental_data["period"] = all_fundamental_data["year"].astype(str) + "-" + all_fundamental_data["month"].astype(str)
all_fundamental_data["ticker_symbol"] = all_fundamental_data["ticker_symbol"].str.replace(".", "-")  # YF idiosyncracy
all_fundamental_data["id"] = all_fundamental_data["ticker_symbol"] + "-" + all_fundamental_data["period"]

# get largest companies according to average market cap
n_largest = 1500
grouped_data = all_fundamental_data.groupby("ticker_symbol").mean()
largest_company_data = grouped_data.sort_values("market_capitalization", ascending=False).head(n_largest)
largest_companies = largest_company_data.index.unique()
largest_companies = [c.replace("-", ".") for c in largest_companies]

# get fundamentals of all largest companies. Aggregation above would have dropped some columns
largest_company_data = all_fundamental_data.loc[all_fundamental_data["ticker_symbol"].isin(largest_companies)]

In [3]:
def tp_tag(row):
    """
    0: rebound
    1: bull
    2: bear
    3: contraction
    """
    if (row["fast_tag"] > 0) & (row["slow_tag"] < 0):
        return 0 
    
    if (row["fast_tag"] > 0) & (row["slow_tag"] > 0):
        return 1 
    
    if (row["fast_tag"] < 0) & (row["slow_tag"] < 0):
        return 2 
    
    if (row["fast_tag"] < 0) & (row["slow_tag"] > 0):
        return 3

In [4]:
# TODO: Add in the 
equity_data = {}

macro_columns = ['unemp_ma_diff', 'confidence', 'MICH', 'bus_credit_change', 'con_credit_change', 
                 'pce_change', 'Adj Close', 'vix_change', 'inverted']

# # norgate params
# price_adjust = norgatedata.StockPriceAdjustmentType.TOTALRETURN 
# padding_setting = norgatedata.PaddingType.NONE   
# start_date = '2020-10-01'  # should change to look up off dataset
# ts_format = 'pandas-dataframe'

for ticker in largest_companies:
    company_data = all_fundamental_data.loc[all_fundamental_data["ticker_symbol"] == ticker]

    if ticker[-1] == "1":
        ticker = ticker[:-1]
        
    if ticker not in price_data["ticker"].unique():
        ticker_obj = yf.Ticker(ticker)
        company_price_data = ticker_obj.history(start='2018-09-01', end='2021-10-01').reset_index()
        
        try:
            company_price_data.columns = ["date_dt", "open", "high", "low", "close", "volume", "div", "splits"]
        except Exception as e:  # occurs if no data
            company_price_data.columns = ["date_dt", "open", "high", "low", "close1", "close", "volume"]
        
        company_price_data["ticker"] = ticker
        
    else:
        company_price_data = price_data.loc[price_data["ticker"] == ticker]

    if company_price_data.shape[0] > 0:
        ### Price Data
        # first, get updated data that we can use to merge with historical data
        try:
            # get columns to align with historical data
            company_price_data["date"] = company_price_data["date_dt"].dt.strftime(date_format="%Y-%m-%d")
            company_price_data["year"] = company_price_data["date_dt"].dt.year
            company_price_data["month"] = company_price_data["date_dt"].dt.month
            company_price_data["ticker"] = ticker
            company_price_data["period"] = company_price_data["year"].astype(str) + "-" + company_price_data["month"].astype(str)
            company_price_data["id"] = company_price_data["ticker"] + "-" + company_price_data["period"]

            # merge
            company_price_data = pd.concat([company_price_data, company_price_data])

        except ValueError as e:
    #         print(f"No new price data available for {ticker}")
            pass

        # get month and year end date for calculating returns
        company_price_data["day"] = company_price_data["date_dt"].dt.day
        company_price_data["day_tag"] = company_price_data["day"].shift(-1) - company_price_data["day"]
        company_price_data["day_tag"] = company_price_data["day_tag"].replace(np.nan, -1)  # in case we get between financial release and end of month

        # get returns and volatility
        company_price_data["rets"] = company_price_data["close"] / company_price_data["close"].shift(1) - 1
        company_price_data["log_rets"] = np.log(1 + company_price_data["rets"])
        company_price_data["volatility"] = company_price_data["log_rets"].rolling(252).std()
        company_price_data["1mo_rets"] = company_price_data["close"] / company_price_data["close"].shift(21) - 1
        company_price_data["1yr_rets"] = company_price_data["close"] / company_price_data["close"].shift(252) - 1

        # compute log and forward returns
        company_price_data["1mo_log_rets"] = np.log(1 + company_price_data["1mo_rets"])

        company_price_data["1mo_fwd_rets"] = company_price_data["close"].shift(-21) / company_price_data["close"] - 1
        company_price_data["1mo_fwd_log_rets"] = np.log(1 + company_price_data["1mo_fwd_rets"])

        # limit to month-end data to align with fundaments
        monthly_data = company_price_data.loc[company_price_data["day_tag"] < 0]

        # get tags for momentum windows
        try:
            monthly_data["fast_tag"] = np.where(monthly_data["1mo_rets"] < 0, -1, 1) 
            monthly_data["slow_tag"] = np.where(monthly_data["1yr_rets"] < 0, -1, 1)
            monthly_data = monthly_data.drop(["1yr_rets"], axis=1)
            monthly_data["tp_tag"] = monthly_data.apply(lambda x: tp_tag(x), axis=1)

            # get turning point, defined as disagreement between slow and fast signals (does this make sense??)
            monthly_data["tp"] = np.where(monthly_data["slow_tag"] != monthly_data["fast_tag"], 1, 0)
            monthly_data["tp_last_yr"] = monthly_data["tp"].rolling(12).sum()

        except Exception as e:
            pass  # no data

        ### Fundamental data
        # and get other fundamental data as well
        company_data = company_data.sort_values("calendar_date")

        # get year, month, day
        company_data["year"] = company_data["date_dt"].dt.year
        company_data["month"] = company_data["date_dt"].dt.month
        company_data["day"] = company_data["date_dt"].dt.day

        # convert to strings for ID
        company_data["year"] = company_data["year"].astype(str)
        company_data["month"] = company_data["month"].astype(str)
        company_data["day"] = company_data["day"].astype(str)

        # get ID for merge
        company_data["period"] = company_data["year"] + "-" + company_data["month"]
        company_data["concat"] = company_data["ticker_symbol"] + "-" + company_data["period"]

        # merge the data
        merged_data = monthly_data.merge(company_data, left_on="id", right_on="id", how="left")
        
        # filter to quarterly data
        merged_data = merged_data[merged_data["month_x"] % 3 ==0]
        merged_data = merged_data.dropna(subset=["volatility"])

        # create standard deviation (variability calcs)
        merged_data["fcf_std"] = merged_data["free_cash_flow_per_share"].rolling(3).std()
        merged_data["capex_std"] = merged_data["capital_expenditure"].rolling(3).std()
        merged_data["earnings_std"] = merged_data['earnings_per_basic_share_usd'].rolling(3).std()
        merged_data["cfo_std"] = merged_data['net_cash_flow_from_operations'].rolling(3).std()
        
        merged_data["fcf_std"] = merged_data["fcf_std"].replace(np.inf, np.nan)
        merged_data["capex_std"] = merged_data["capex_std"].replace(np.inf, np.nan)
        merged_data["earnings_std"] = merged_data["earnings_std"].replace(np.inf, np.nan)
        merged_data["cfo_std"] = merged_data["cfo_std"].replace(np.inf, np.nan)

        merged_data["avg_assets"] = (merged_data["total_assets"] + merged_data["total_assets"].shift(1)) / 2
        merged_data["avg_equity"] = (merged_data["shareholders_equity"] + merged_data["shareholders_equity"].shift(1)) / 2
        merged_data["avg_wc"] = (merged_data["working_capital"] + merged_data["working_capital"].shift(1)) / 2
        merged_data["avg_debt"] = (merged_data["total_debt"] + merged_data["total_debt"].shift(1)) / 2
        
        # setup for targets later
        merged_data["fwd_fcf_per_share"] = merged_data["free_cash_flow_per_share"].shift(-1)
#         merged_data["fwd_eps"] = merged_data["earnings_per_basic_share_usd"].shift(-1)

        # Fill first 3 entries, as those will have NaNs from Standard Deviation calcs ***
        merged_data["fcf_std"] = merged_data["fcf_std"].fillna(merged_data["fcf_std"].median())
        merged_data["cfo_std"] = merged_data["cfo_std"].fillna(merged_data["cfo_std"].median())
        merged_data["capex_std"] = merged_data["capex_std"].fillna(merged_data["capex_std"].median())
        merged_data["earnings_std"] = merged_data["capex_std"].fillna(merged_data["earnings_std"].median())
        merged_data["avg_assets"] = merged_data["avg_assets"].fillna(merged_data["avg_assets"].median())
        merged_data["avg_equity"] = merged_data["avg_equity"].fillna(merged_data["avg_equity"].median())
        merged_data["avg_wc"] = merged_data["avg_wc"].fillna(merged_data["avg_wc"].median())
        merged_data["avg_debt"] = merged_data["avg_debt"].fillna(merged_data["avg_debt"].median())
        merged_data["avg_capital"] = merged_data["avg_debt"] + merged_data["avg_equity"]

        # if label is Nan just drop
        equity_data[ticker] = merged_data.dropna(subset=["fwd_fcf_per_share"])


- BRK.B: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
- MOB: No data found for this date range, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
- WYE: No data found for this date range, symbol may be delisted
- TFCF: No data found, symbol may be delisted
- WLA: No data found for this date range, symbol may be delisted
- TYC: No data found for this date range, symbol may be delisted
- CPQ: No data found for this date range, symbol may be delisted
- KRFT: No data found for this date range, symbol may be delisted
- USW: No data found for this date range, symbol may be delisted
- JAVA: Data doesn't exist for startDate = 1535785200, endDate = 1633071600
- CELG: No data found, symbol may be delisted
- FBF: No data found for this date range, symbol may be delisted
- TWC: No data found for this date range, symbol may be delisted
- DTV: No data found, symbol may be delisted
- INF.A: No data found, 

- CAM: No data found for this date range, symbol may be delisted
- TLAB: No data found for this date range, symbol may be delisted
- UPCOY: No data found for this date range, symbol may be delisted
- CFN: No data found for this date range, symbol may be delisted
- BT: No data found, symbol may be delisted
- MI: No data found for this date range, symbol may be delisted
- MFNXQ: No data found for this date range, symbol may be delisted
- CPN: No data found for this date range, symbol may be delisted
- SDLI: No data found for this date range, symbol may be delisted
- SIAL: No data found for this date range, symbol may be delisted
- ABI: No data found for this date range, symbol may be delisted
- CITGQ: No data found, symbol may be delisted
- NFB: No data found for this date range, symbol may be delisted
- CF2: No data found, symbol may be delisted
- PPW: No data found for this date range, symbol may be delisted
- VMED: No data found for this date range, symbol may be delisted
- HDS: No da

- OK: No data found for this date range, symbol may be delisted
- ZU: No data found, symbol may be delisted
- FEYE: No data found, symbol may be delisted
- WCG: No data found, symbol may be delisted
- AAMRQ: No data found for this date range, symbol may be delisted
- NCF: No data found for this date range, symbol may be delisted
- CDWC: No data found for this date range, symbol may be delisted
- PTHN: No data found for this date range, symbol may be delisted
- CTX: No data found for this date range, symbol may be delisted
- FAM: No data found for this date range, symbol may be delisted
- APCC: No data found for this date range, symbol may be delisted
- RLR: No data found for this date range, symbol may be delisted
- LPT: No data found, symbol may be delisted
- GMCR: No data found for this date range, symbol may be delisted
- NTLDQ: No data found, symbol may be delisted
- PFPT: No data found, symbol may be delisted
- DST: No data found for this date range, symbol may be delisted
- CLWR:

In [5]:
# probably merge macro data after everything else is done
equity_df = pd.concat([equity_data[t] for t in equity_data])
equity_df = equity_df.dropna(subset=["concat"])
equity_df["quarter"] = equity_df["date_dt_x"].dt.quarter
equity_df["quarter_end"] = equity_df["quarter"] % 3
equity_df = equity_df.merge(sharadar_sectors[["ticker", "sicsector"]], left_on="ticker_symbol", right_on="ticker")

# drop any duplicates
equity_df = equity_df[~equity_df["id"].duplicated()]

equity_df.shape


(46003, 162)

In [6]:
# write to CSV
equity_df.to_csv("C:/Users/lbianculli/dev/us_equities/ML_categorization/all_fundamentals.csv")


### final_preprocessing file next